In [1]:
# Import libraries
%matplotlib inline
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
#from sklearn.pipeline import FeatureUnion

In [2]:
# Importa os dados
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataset = pd.read_csv('pima-indians-diabetes.data.csv')
dataset.columns=names
dataset.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [3]:
# Distribuição da variável target
print(Counter(dataset['class']))

Counter({0: 500, 1: 267})


In [4]:
# Encontra colunas com valores ausentes e apresenta um resumo da base
columnsDF = pd.DataFrame(dataset.isnull().any(), columns = np.array(['hasMissing']))
print(columnsDF.query('hasMissing == True'))

# Encontra as linhas com algum valor ausente
null_data = dataset[dataset.isnull().any(axis=1)]
print(null_data.shape[0])

# Elimina as observações com valores ausentes
datasetFiltered = dataset.dropna()

Empty DataFrame
Columns: [hasMissing]
Index: []
0


In [5]:
# Extrai a variável target para uma variável específica e a elimina da base 
target = datasetFiltered['class']
datasetFiltered = datasetFiltered.drop('class', 1)

In [6]:
# Configura alguns parâmetros
seed = 7
test_size = 0.25

In [7]:
# Separa a base entre treino e teste de forma aleatória
X = datasetFiltered.values
Y = target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

In [8]:
# Agora vamos criar o modelo usando pipelines. 
# A ideia do pipeline é "empilhar" todas as etapas do modelo, desde a transformação dos dados até a execução do algoritmo de ML.
# Neste exemplo, nosso modelo é composto por duas etapas: 
# - primeiro reduzimos a dimensionalidade dos dados extraindo os componentes principais (PCA)
# - em seguida definimos que queremos usar um algoritmo de árvore de decisão
estimators = []
estimators.append(('pca', PCA(n_components=2)))
estimators.append(('model', DecisionTreeClassifier(random_state = seed)))
baseline = Pipeline(estimators)
baseline.fit(X_train, Y_train)

# Durante o treino, o primeiro passo será executar a análise dos componentes principais (PCA). Já o segundo passo será 
# o treinamento do modelo de árvore de decisão, sendo que as variáveis utilizadas no modelo (features) serão os 
# dois componentes principais extraídos na etapa 1

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('model', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=7,
            splitter='best'))])

In [9]:
# Testa contra os dados de teste
predictions = baseline.predict(X_test)
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))

[[91 34]
 [33 34]]
             precision    recall  f1-score   support

          0       0.73      0.73      0.73       125
          1       0.50      0.51      0.50        67

avg / total       0.65      0.65      0.65       192



In [10]:
# Acessando as etapas do pipeline
# Você pode usar o próprio pipeline para fazer as predições nos dados de produção.
# Porém, também é possível acessar cada componente do pipeline da seguinte forma
selector = baseline.named_steps['pca']
model = baseline.named_steps['model']